<img src="https://raw.githubusercontent.com/dimitreOliveira/MachineLearning/master/Kaggle/Microsoft%20Malware%20Prediction/Microsoft_logo.png" width="600" height="170">
<h1><center>Microsoft Malware Prediction</center></h1>
<h2><center>Can you predict if a machine will soon be hit with malware?</center></h2>

### Content:
* [Exploratory Data Analysis [1st part, raw data]](#Exploratory-Data-Analysis-[1st-part,-raw-data])
* [Pre process](#Pre-process)
* [Exploratory Data Analysis [2nd part, clean data]](#Exploratory-Data-Analysis-[2nd-part,-clean-data])
* [Process data for LGB model](#Process-data-for-LGB-model)
* [Model training](#Model-training)
* [Predictions](#Predictions)
* [Results](#Output-results)

#### Checkout the second part of this EDA on [this kernel](https://www.kaggle.com/dimitreoliveira/malware-detection-extended-eda/notebook)
#### You can also find this on [GitHub here](https://github.com/dimitreOliveira/MachineLearning/blob/master/Kaggle/Microsoft%20Malware%20Prediction/Malware%20Detection%20-%20EDA%20and%20LGBM.ipynb)

### Dependencies

In [ ]:
import dask
import dask.dataframe as dd
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, mean_squared_error, classification_report

%matplotlib inline
sns.set(style="whitegrid")
warnings.filterwarnings("ignore")
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

### Load data

In [ ]:
train = dd.read_csv('../input/train.csv', dtype=dtypes)
train = train.compute()

In [ ]:
def update_feature_lists():
    binary = [c for c in train.columns if train[c].nunique() == 2]
    numerical = ['Census_ProcessorCoreCount',
                 'Census_PrimaryDiskTotalCapacity',
                 'Census_SystemVolumeTotalCapacity',
                 'Census_TotalPhysicalRAM',
                 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
                 'Census_InternalPrimaryDisplayResolutionHorizontal',
                 'Census_InternalPrimaryDisplayResolutionVertical',
                 'Census_InternalBatteryNumberOfCharges']
    categorical = [c for c in train.columns if (c not in numerical) & (c not in binary)]
    return binary, numerical, categorical
    
binary_columns, true_numerical_columns, categorical_columns = update_feature_lists()

### Raw dataset overview

In [ ]:
print('Dataset number of records: %s' % train.shape[0])
print('Dataset number of columns: %s' % train.shape[1])
train.head()

## Exploratory Data Analysis [1st part, raw data]
### Column type distribution

In [ ]:
total = train.shape[0]
missing_df = []
cardinality_df = []
for col in train.columns:
    missing_df.append([col, train[col].count(), total])
    cardinality = train[col].nunique()
    if cardinality > 2 and col != 'MachineIdentifier':
        cardinality_df.append([col, cardinality])
    
missing_df = pd.DataFrame(missing_df, columns=['Column', 'Number of records', 'Total']).sort_values("Number of records", ascending=False)
cardinality_df = pd.DataFrame(cardinality_df, columns=['Column', 'Cardinality']).sort_values("Cardinality", ascending=False)
type_df = [['Binary columns', len(binary_columns)], ['Numerical columns', len(true_numerical_columns)], ['Categorical columns', len(categorical_columns)]]
type_df = pd.DataFrame(type_df, columns=['Type', 'Column count']).sort_values('Column count', ascending=True)

In [ ]:
f, ax = plt.subplots(figsize=(7, 7))
sns.barplot(x="Column count", y="Type", data=type_df, label="Missing", palette='Spectral')
plt.show()

As we can see, there are far more categorical columns than numerical or binary.

### Null values count

In [ ]:
f, ax = plt.subplots(figsize=(10, 15))
sns.set_color_codes("pastel")
sns.barplot(x="Total", y="Column", data=missing_df, label="Missing", color="orange")
sns.barplot(x="Number of records", y="Column", data=missing_df, label="Existing", color="b")
ax.legend(ncol=2, loc="upper right", frameon=True)
plt.show()

 For most of the feature we don't have too many missing values to worry about, but we have 7 features with more than 50% missing values, probably would be a good idea to remove them.

### Column cardinality
* Only shown columns with cardinalities greater than 2

In [ ]:
f, ax = plt.subplots(figsize=(10, 15))
sns.set_color_codes("pastel")
sns.barplot(x="Cardinality", y="Column", data=cardinality_df, label="Existing", color="red")
plt.show()

Some of the features have too many categorical values (high cardinality), this can be a problem to some models, if you one-hot encode them you will end up with features too sparse, as this is a simple iteration I'll remove those features.

### Label count

In [ ]:
f, ax = plt.subplots(figsize=(6, 6))
ax = sns.countplot(x="HasDetections", data=train, label="Label count")
sns.despine(bottom=True)

Fortunately our label is balanced, this will make things easier for us.

## Pre process
### Clean data
#### Remove columns with high cardinality (more than 500 categories)

In [ ]:
high_cardinality_columns = [c for c in categorical_columns if train[c].nunique() > 500]
high_cardinality_columns.remove('MachineIdentifier')  # Remove ID
train.drop(high_cardinality_columns, axis=1, inplace=True)
print('Columns with high cardinality: ', high_cardinality_columns)

#### Remove columns with more than 40% missing data

In [ ]:
high_null_columns = [c for c in train.columns if train[c].count() < len(train)*0.6]
train.drop(high_null_columns, axis=1, inplace=True)
print('Columns with more than 40% null values: ', high_null_columns)

#### Remove unwanted columns

In [ ]:
unwanted_columns = ['MachineIdentifier']
train.drop(unwanted_columns, axis=1, inplace=True)

In [ ]:
# Remove rows from numeric features with missing values (need this to plot distributions)
train.dropna(subset=true_numerical_columns, inplace=True)

## Exploratory Data Analysis [2nd part, clean data]

### Correlate version and build features with the label

I had an initial hypothesis that devices with older versions or builds would have more malware, since their vulnerabilities probably would be more well known, and could be used for malicious ends.

In [ ]:
binary_columns, true_numerical_columns, categorical_columns = update_feature_lists()

In [ ]:
def plot_label_distribution():
    for feature in (binary_columns + categorical_columns):
        if 'Version' in feature and feature != 'Census_OSVersion':
            sns.catplot(data=train, x=feature, col="HasDetections", kind="count", height=8).set_xticklabels(rotation=90)
            sns.despine(left=True)
            plt.tight_layout()

plot_label_distribution()

### About the features related to Version
* EngineVersion and AppVersion: Both distributions seems to be almost the same.

Also as you can see newer versions have less data.

In [ ]:
def plot_label_distribution():
    for feature in (binary_columns + categorical_columns):
        if 'Build' in feature and feature != 'OsBuildLab':
            sns.catplot(data=train, x=feature, col="HasDetections", kind="count", height=8).set_xticklabels(rotation=90)
            sns.despine(left=True)
            plt.tight_layout()

plot_label_distribution()

### About the features related to build
* OsBuild and Census_OSBuildNumber: Not much going on here, seems to be something close to 50% malware detection and 50% with no malware detection.
* Census_OSBuildRevision: Here things are more interesting, some categories have a similar label count, but some of them have a lot more of one label than the other.

**About my hypothesis: With those visualizations it's hard to validate or drop it, we don't have enough data on more recent version or build features to say if they are more secure than older ones.**

### Numeric columns distribution on the whole set and by label

In [ ]:
def plot_distribution():
    for feature in true_numerical_columns:
        f, axes = plt.subplots(1, 3, figsize=(20, 8), sharex=True)
        sns.distplot(train[feature], ax=axes[0]).set_title("Complete set")
        sns.distplot(train[train['HasDetections']==1][feature], ax=axes[1]).set_title("HasDetections = 1")
        sns.distplot(train[train['HasDetections']==0][feature], ax=axes[2]).set_title("HasDetections = 0")
        sns.despine(left=True)
        plt.tight_layout()

plot_distribution()

### About the features related to build
* Census_ProcessorCoreCount: Malware detection seems to more skewed towards right and non-detection are more concentrated on the first 10 values.
* Census_PrimaryDiskTotalCapacity: The distributions seems to be the same.
* Census_SystemVolumeTotalCapacity and Census_TotalPhysicalRAM: Malware detection seems to more concentrated on the beginning and non-detection seems to more skewed towards right.
* Census_InternalPrimaryDiagonalDisplaySizeInInches: The distributions are very similar but non-detection have a longer right tail.
* Census_InternalPrimaryDisplayResolutionHorizontal: The distributions seems to be the same.
* Census_InternalPrimaryDisplayResolutionVertical: The distributions are very similar but non-detection have a longer right tail.
* Census_InternalBatteryNumberOfCharges: The distributions seems to be the same.

In [ ]:
train.head()

In [ ]:
train[true_numerical_columns].describe().T

## Process data for LGB model

In [ ]:
# Remove rows with NA
train.dropna(inplace=True)

### Train/validation random split (85% train / 15% validation)

In [ ]:
# Get labels
labels = train['HasDetections']
train.drop('HasDetections', axis=1, inplace=True)

X_train, X_val, Y_train, Y_val = train_test_split(train, labels, test_size=0.15,random_state=1)

In [ ]:
binary_columns, true_numerical_columns, categorical_columns = update_feature_lists()

# Label encoder
indexer = {}
for col in categorical_columns:
    _, indexer[col] = pd.factorize(X_train[col])
    
for col in categorical_columns:
    X_train[col] = indexer[col].get_indexer(X_train[col])
    X_val[col] = indexer[col].get_indexer(X_val[col])

In [ ]:
params = {'num_leaves': 60,
         'min_data_in_leaf': 100, 
         'objective':'binary',
         'max_depth': -1,
         'learning_rate': 0.1,
         "boosting": "gbdt",
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8 ,
         "bagging_seed": 1,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "random_state": 133,
         "verbosity": -1}

In [ ]:
lgb_train = lgb.Dataset(X_train, label=Y_train)
lgb_val = lgb.Dataset(X_val, label=Y_val)

## Model training

In [ ]:
model = lgb.train(params, lgb_train, 10000, valid_sets=[lgb_train, lgb_val], early_stopping_rounds=200, verbose_eval=100)

### Model feature importance

In [ ]:
lgb.plot_importance(model, figsize=(15, 10))
plt.show()

In [ ]:
train_predictions_raw = model.predict(X_train, num_iteration=model.best_iteration)
val_predictions_raw = model.predict(X_val, num_iteration=model.best_iteration)

train_predictions = np.around(train_predictions_raw)
val_predictions = np.around(val_predictions_raw)

### Model metrics

In [ ]:
target_names=['HasDetections = 0', 'HasDetections = 1']
print('-----Train-----')
print(classification_report(Y_train, train_predictions, target_names=target_names))
print('-----Validation-----')
print(classification_report(Y_val, val_predictions, target_names=target_names))

### Confusion matrix

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(16, 5), sharex=True)
train_cnf_matrix = confusion_matrix(Y_train, train_predictions)
val_cnf_matrix = confusion_matrix(Y_val, val_predictions)

train_cnf_matrix_norm = train_cnf_matrix / train_cnf_matrix.sum(axis=1)[:, np.newaxis]
val_cnf_matrix_norm = val_cnf_matrix / val_cnf_matrix.sum(axis=1)[:, np.newaxis]

train_df_cm = pd.DataFrame(train_cnf_matrix_norm, index=[0, 1], columns=[0, 1])
val_df_cm = pd.DataFrame(val_cnf_matrix_norm, index=[0, 1], columns=[0, 1])

sns.heatmap(train_df_cm, annot=True, fmt='.2f', cmap="Blues", ax=axes[0]).set_title("Train")
sns.heatmap(val_df_cm, annot=True, fmt='.2f', cmap="Blues", ax=axes[1]).set_title("Validation")
plt.show()

### Probabilities distribution
* One thing that is often overlooked is the output probability from the models, this can help you see how confident your model is about it's predictions.
* From your model probability distribution what you would like to see is a higher distribution on the extremes that would mean high probability of class 0 or 1, lots of values on the middle can mean that your model don't high confidence about it's predictions.

In [ ]:
f, ax = plt.subplots(figsize=(24, 6))
sns.set_color_codes("pastel")
ax = sns.distplot(train_predictions_raw, color="blue", kde_kws={"label": "Train"}, axlabel='Probability distribution')
ax = sns.distplot(val_predictions_raw, color="orange", kde_kws={"label": "Validation"})
sns.despine(left=True)

## Predictions

### Load test set

In [ ]:
# Because of memory issues I'm loading only part of test set.
test = dd.read_csv('../input/test.csv', dtype=dtypes, usecols=(['MachineIdentifier'] + list(X_train.columns))).head(n=1000000)

### Pre process test

In [ ]:
submission = pd.DataFrame({"MachineIdentifier":test['MachineIdentifier']})
test.drop('MachineIdentifier', axis=1, inplace=True)

for col in categorical_columns:
    test[col] = indexer[col].get_indexer(test[col])

### Make predictions

In [ ]:
predictions = model.predict(test, num_iteration=model.best_iteration)

## Output results

In [ ]:
submission["HasDetections"] = predictions
submission.to_csv("submission.csv", index=False)
submission.head(10)